In [1]:
import math
import numpy as np
from random import choice
from matplotlib import pyplot as plt

### Montgomery Form

In [85]:
def MForm(a,n):
    exp = math.ceil(math.log(n, 2))
    R = pow(2, exp)
    return (a*R)%n

In [86]:
MForm(2,11)

10

In [87]:
MForm(5,161)

153

## MMM

In [88]:
def MMM(a,b,p):
    return MForm(a*b, p)

In [89]:
MMM(2,5,7)

3

In [58]:
MMM(4,9,161)

39

In [90]:
MMM(9,7,15)

3

## MMM reduction

In [91]:
def inv(a, n):
    for i in range(1, n):
        if (a*i)%n == 1:
            return i

In [102]:
def find_v(n):
    exp = math.ceil(math.log(n, 2))
    R = pow(2, exp)
    Rinv = inv(R, n)
    v = (R*Rinv-1)//n
    return (R, v)

In [107]:
def MMMr(a, b, n):
    flag = 0
    D = MForm(a, n)*MForm(b, n)
    R, v = find_v(n)
    m = ((D%R)*v)%R
    U = (D+(m*n))//R # 0 <= U <= 2n
    if U >= n:
        U = U-n
        flag = 1
    return (U, flag)

In [108]:
MMMr(2,5,7)

(3, 0)

In [109]:
MMMr(4,9,161)

(39, 0)

In [110]:
MMMr(9,7,15)

(3, 1)

##  Left-to-right Montgomery ladder with MM algorithm

In [137]:
def LtRMontLadder(m, kb, n):
    exp = math.ceil(math.log(n, 2))
    R = pow(2, exp)
    Rinv = inv(R, n)
    R0 = MMMr(m, Rinv**2, n)[0]
    R1 = MMMr(R0, R0, n)[0]
    for i in range(3,len(kb)):
        if kb[i] == '0':
            R1 = MMMr(R0, R1, n)[0]
            R0 = MMMr(R0,R0,n)[0]
            w.append(MMMr(R0, R1, n)[1])
            w.append(MMMr(R0,R0,n)[1])
        else:
            R0 = MMMr(R0, R1, n)[0]
            R1 = MMMr(R1,R1,n)[0]
            w.append(MMMr(R0, R1, n)[1])
            w.append(MMMr(R1, R1, n)[1])
    return (R0*R)%n

In [138]:
w = []
p = 143
kb = bin(23)
print(kb[2:])
m = 3
print(LtRMontLadder(m, kb, p))
print(w)

10111
126
[0, 0, 1, 0, 0, 0, 1, 0]


In [139]:
(m**23)%p

126

In [140]:
w = []
p = 4277
kb = bin(11)
print(kb[2:])
m = 12
print(LtRMontLadder(m, kb, p))
print(w)

1011
1130
[0, 0, 0, 0, 0, 0]


In [141]:
(m**11)%p

1130

## Attack

In [21]:
def convert(L):
    '''
    From binary to decimal
    '''
    
    r = 0
    for i in range(len(L)-1,-1,-1):
        if L[i] == 1:
            r += pow(2,len(L)-i-1)
    return r

In [22]:
def uM(ER):
    N = len(ER)
    l = len(ER[0])
    l0 = l//2
    pR_E = 0
    for i in range(N):
        for j in range(0, l, 2):
            pR_E += ER[i][j]
    return pR_E/(N*l0)

def uQ(ER):
    N = len(ER)
    l = len(ER[0])
    l0 = l//2
    pn_E = 0
    for i in range(N):
        for j in range(1, l, 2):
            pn_E += ER[i][j]
    return pn_E/(N*l0)

In [23]:
print(w)
print(uM([w,[1, 1, 0, 0, 0, 1, 0, 0, 0, 0],[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]))
print(uQ([w,[1, 1, 0, 0, 0, 1, 0, 0, 0, 0],[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]))

[0, 0, 0, 0, 0, 0]
0.2222222222222222
0.3333333333333333


In [24]:
def p_Estimators(ER):
    um = uM(ER)
    uq = uQ(ER)
    pR_e = ((12*uq)-(12*um))/(1+(6*uq)-(8*um))
    pn_e = (4*um)-(3*uq)
    return (1/pR_e,pn_e)

In [25]:
N = []
s = (0,1)
for j in range(10):
    for i in range(100+(j*100)):
        N.append([choice(s),choice(s),choice(s),choice(s)])
    print(p_Estimators(N))

(0.22222222222222274, 0.48)
(0.16666666666667038, 0.4933333333333332)
(0.08333333333334443, 0.4983333333333331)
(3.499999999999334, 0.5105000000000002)
(0.634703196347032, 0.5926666666666669)
(0.5967741935483852, 0.5528571428571429)
(0.6043956043956031, 0.5589285714285717)
(0.6022727272727273, 0.5662499999999999)
(0.5650793650793665, 0.5395555555555556)
(0.5669291338582698, 0.5392727272727273)


In [26]:
p_Estimators([w,[1, 1, 1, 0, 0, 1, 0, 0, 0, 0],[1, 1, 0, 0, 1, 0, 0, 0, 1, 0]])

(0.41666666666666663, 0.7777777777777777)

In [27]:
def vErrors(u):
    r = []
    for i in range(pow(2,2*u)):
        bf = list(bin(i)[2:])
        temp = [0]*((2*u)-len(bf))+[eval(i) for i in bf]
        if temp not in r:
            r.append(temp)
    return r
vErrors(2)

[[0, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 1],
 [0, 1, 0, 0],
 [0, 1, 0, 1],
 [0, 1, 1, 0],
 [0, 1, 1, 1],
 [1, 0, 0, 0],
 [1, 0, 0, 1],
 [1, 0, 1, 0],
 [1, 0, 1, 1],
 [1, 1, 0, 0],
 [1, 1, 0, 1],
 [1, 1, 1, 0],
 [1, 1, 1, 1]]

In [28]:
def ham(V):
    c = 0
    for i in V:
        if i == 1:
            c += 1
    return c

In [29]:
def prob_u2(vt, ER, pR):
    prob = 0
    if vt == [0, 0] or vt == [1,1]:
        if ER == [0,0,0,0]:
            return 1+(1/264)*(pR**8)-(1/72)*(pR**6)-(11/336)*(pR**5)+(17/504)*(pR**4)+(7/90)*(pR**3)+(1/3)*(pR**2)-(7/6)*(pR)
        elif ER == [0,0,0,1]:
            return -(1/264)*(pR**8)+(1/72)*(pR**6)+(1/84)*(pR**5)+(1/72)*(pR**4)-(7/90)*(pR**3)+(1/8)*(pR**2)+(1/4)*pR
        elif ER == [0,0,1,0]:
            return -(1/264)*(pR**8)+(1/72)*(pR**6)+(1/48)*(pR**5)-(17/504)*(pR**4)-(5/24)*(pR**2)+(1/3)*pR
        elif ER == [0,0,1,1]:
            return (1/264)*(pR**8)-(1/72)*(pR**6)-(1/72)*(pR**4)+(1/8)*(pR**2)
        elif ER == [0,1,0,0]:
            return -(1/264)*(pR**8)+(11/336)*(pR**5)+(1/72)*(pR**4)-(1/36)*(pR**3)-(5/24)*(pR**2)+(1/4)*(pR)
        elif ER == [0,1,0,1]:
            return (1/264)*(pR**8)-(1/84)*(pR**5)-(1/72)*(pR**4)+(1/36)*(pR**3)
        elif ER == [0,1,1,0]:
            return (1/264)*(pR**8)-(1/48)*(pR**5)-(1/72)*(pR**4)+(1/12)*(pR**2)
        elif ER == [0,1,1,1]:
            return -(1/264)*(pR**8)+(1/72)*(pR**4)
        elif ER == [1,0,0,0]:
            return (1/7)*((1/12)*(pR**5)-(1/3)*(pR**4))-(1/264)*(pR**8)+(1/72)*(pR**6)+(1/48)*(pR**5)-(1/20)*(pR**3)-(1/8)*(pR**2)+(1/3)*pR
        elif ER == [1,0,0,1]:
            return (1/264)*(pR**8)-(1/72)*(pR**6)-(1/84)*(pR**5)+(1/20)*(pR**3)
        elif ER == [1,0,1,0]:
            return (1/264)*(pR**8)-(1/72)*(pR**6)-(1/48)*(pR**5)+(1/21)*(pR**4)
        elif ER == [1,0,1,1]:
            return -(1/360)*(pR**8)+(1/72)*(pR**5)
        elif ER == [1,1,0,0]:
            return (1/264)*(pR**8)-(11/336)*(pR**5)+(1/8)*(pR**2)
        elif ER == [1,1,0,1]:
            return -(1/360)*(pR**8)+(1/72)*(pR**5)
        elif ER == [1,1,1,0]:
            return -(1/264)*(pR**8)+(1/84)*(pR**5)
        else:
            return (1/264)*(pR**8)
    elif vt == [0,1] or vt == [1,0]:
        if ER == [0,0,0,0]:
            return 1+(1/360)*(pR**8)-(1/200)*(pR**6)-(13/504)*(pR**5)+(1/240)*(pR**4)+(7/90)*(pR**3)+(13/36)*(pR**2)-(7/6)*(pR)
        elif ER == [0,0,0,1]:
            return -(1/360)*(pR**8)+(1/200)*(pR**6)+(1/72)*(pR**5)+(1/240)*(pR**4)-(17/72)*(pR**2)+(1/3)*pR
        elif ER == [0,0,1,0]:
            return -(1/360)*(pR**8)+(1/200)*(pR**6)+(1/84)*(pR**5)+(1/40)*(pR**4)-(7/90)*(pR**3)-(1/8)*(pR**2)+(1/4)*pR
        elif ER == [0,0,1,1]:
            return (1/360)*(pR**8)-(1/200)*(pR**6)-(1/40)*(pR**4)+(1/8)*(pR**2)
        elif ER == [0,1,0,0]:
            return -(1/360)*(pR**8)+(1/200)*(pR**6)+(13/504)*(pR**5)-(1/48)*(pR**4)-(1/36)*(pR**3)-(1/8)*(pR**2)+(1/4)*(pR)
        elif ER == [0,1,0,1]:
            return (1/360)*(pR**8)-(1/200)*(pR**6)-(1/72)*(pR**5)+(1/48)*(pR**4)
        elif ER == [0,1,1,0]:
            return (1/360)*(pR**8)-(1/200)*(pR**6)-(1/84)*(pR**5)+(1/36)*(pR**3)
        elif ER == [0,1,1,1]:
            return -(1/360)*(pR**8)+(1/200)*(pR**6)
        elif ER == [1,0,0,0]:
            return -(1/360)*(pR**8)+(13/504)*(pR**5)+(1/40)*(pR**4)-(1/20)*(pR**3)-(17/72)*(pR**2)+(1/3)*pR
        elif ER == [1,0,0,1]:
            return (1/360)*(pR**8)-(1/72)*(pR**5)-(1/40)*(pR**4)+(1/9)*(pR**2)
        elif ER == [1,0,1,0]:
            return (1/360)*(pR**8)-(1/84)*(pR**5)-(1/40)*(pR**4)+(1/20)*(pR**3)
        elif ER == [1,0,1,1]:
            return -(1/360)*(pR**8)+(1/40)*(pR**4)
        elif ER == [1,1,0,0]:
            return (1/360)*(pR**8)-(13/504)*(pR**5)+(1/8)*(pR**2)
        elif ER == [1,1,0,1]:
            return -(1/360)*(pR**8)+(1/72)*(pR**5)
        elif ER == [1,1,1,0]:
            return -(1/360)*(pR**8)+(1/84)*(pR**5)
        else:
            return (1/360)*(pR**8)

In [30]:
prob_u2([1,0], w[:4], 0.9)

0.2852413943928571

In [31]:
def Lsum_mod2(L1,L2):
    L = []
    for i in range(len(L1)):
        L.append(L1[i]+L2[i])
    return L

In [32]:
def Attack_u2(ER):
    # u = 2
    t_est = []
    pR_E, pn_E = p_Estimators(ER)
    prob_l = []
    Err_v = vErrors(2)
    # i = u-1, 2(u-1), 3(u-1)...
    for theta in [[0,0],[0,1],[1,0],[1,1]]:
        prob_temp = 0
        for v in Err_v:
            for er_v in ER:
                for j in range(1,len(er_v),2):
                    prob_temp += prob_u2(theta, Lsum_mod2(v,er_v), pR_E)
            prob_temp *= pow(pn_E, ham(v))*pow(1-pn_E,4-ham(v))
        prob_l.append(prob_temp)
    for i in range(0,len(ER[0]),2):
        Accum = [0]*16
        for v_n in ER:
            Accum[convert(v_n[i:i+4])] += 1
        for theta in [[0,0],[1,0]]: # u//2
            Tt = 0
            for vec in Err_v:
                Tt += Accum[convert(vec)]*np.log(prob_u2(theta, vec, pR_E))
            t_est.append(Tt)
    k_est = [0]*(len(ER[0])//2)
    k_est[0] = 1
    k_est[-1] = 1
    for i in range(len(ER[0])//2 - 2):
        if i == 0:
            k_est[1] = int((t_est[0] + k_est[0])%2)
        else:
            for j in range(2):
                #print(i+j)
                k_est[1+i-j] = int((t_est[i+j]+k_est[i]))%2
    k_est.reverse()
    return (k_est, convert(k_est))

In [33]:
Attack_u2([[1,0,1,1,0,1,0,1,1,1],[1,1,0,0,1,1,0,1,0,1],[1,1,0,1,1,1,1,1,0,1],[1,1,0,1,1,0,1,1,0,0]])

([1, 0, 0, 0, 1], 17)

In [34]:
Attack_u2([w,[0, 1, 1, 0, 1, 0, 1, 0, 0, 0],[0,0,1,0,1,0,0,1,0,1]])

([1, 0, 1], 5)

In [35]:
Attack_u2([w,[1, 1, 1, 0, 0, 1, 0, 0, 0, 0],[1, 1, 0, 0, 1, 0, 0, 0, 1, 0]])

([1, 1, 1], 7)